In [12]:
import os
import sys
import torch
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.optim as optim
import math
from tensorboardX import SummaryWriter

import import_ipynb
from CustomDataset import ControlsDataset
from Model import ConvNet

REPORT_EVERY_ITER = 20

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('using device', device)

('using device', device(type='cpu'))


In [13]:
dataset = ControlsDataset()
print("Stack size", dataset.stack_size)
# dataloader = DataLoader(dataset, batch_size = 256, shuffle = True, num_workers = 0)
net = ConvNet().to(device)

print("Data size", dataset.data_frame.size)
print("number of parameters: ", sum(p.numel() for p in net.parameters()))

('Stack size', 1)
('units after conv', 512)
('conv parameters: ', 168224)
('fc parameters: ', 164353)
('Data size', 11018)
('number of parameters: ', 332577)


In [14]:
torch.cuda.empty_cache()

In [17]:
validation_split = .2
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
# if shuffle_dataset :
#     np.random.seed(random_seed)
#     np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)


# Training data loader
dataloader = DataLoader(dataset, batch_size = 256, num_workers = 0, sampler=train_sampler)

# Validation data loader
validLoader = DataLoader(dataset, batch_size = 256, num_workers = 0, sampler=valid_sampler)

print(len(dataloader))
print(len(validLoader))
# train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
#                                            sampler=train_sampler)
# validation_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
#                                                 sampler=valid_sampler)

# dataloader = DataLoader(dataset, batch_size = 256, shuffle = True, num_workers = 0)

35
9


In [ ]:
epochs = 25
optimizer = optim.Adam(net.parameters(), lr=0.0001)
criterion = nn.MSELoss()
writer = SummaryWriter()
iter_no = 0

torch.cuda.empty_cache()
for epoch in range(epochs):
    for i_batch, sampled_batch in enumerate(dataloader):
        iter_no += 1
        images = sampled_batch['image'].to(device).float()
        controls = sampled_batch['control'].to(device).float()
    
        optimizer.zero_grad()
        prediction = net(images)
        #print((prediction[0], controls[0]))
        loss = criterion(prediction, controls)
        loss.backward()
        optimizer.step()

        if iter_no % REPORT_EVERY_ITER == 0:
            writer.add_scalar("Loss", loss.item(), iter_no)
            torch.save(net.state_dict(), "snapshots/saved_model_{}".format(loss.item()))
        
        out = "{0},{1}\tLoss:{2}\tAllocated:{3}GB\tCached:{4}GB\n"
        print(out.format(str(epoch),
                        str(iter_no),
                        round(loss.item(),5),
                         'na', 'na'
#                         round(torch.cuda.memory_allocated(0)/1024**3,3),
#                         round(torch.cuda.memory_allocated(0)/1024**3,3)
                        ))
        torch.cuda.empty_cache()

0,1	Loss:0.05935	Allocated:naGB	Cached:naGB

0,2	Loss:0.04808	Allocated:naGB	Cached:naGB

0,3	Loss:0.0698	Allocated:naGB	Cached:naGB

0,4	Loss:0.0435	Allocated:naGB	Cached:naGB

0,5	Loss:0.06472	Allocated:naGB	Cached:naGB

0,6	Loss:0.05344	Allocated:naGB	Cached:naGB

0,7	Loss:0.05261	Allocated:naGB	Cached:naGB

0,8	Loss:0.04632	Allocated:naGB	Cached:naGB

0,9	Loss:0.04518	Allocated:naGB	Cached:naGB

0,10	Loss:0.05029	Allocated:naGB	Cached:naGB

0,11	Loss:0.04766	Allocated:naGB	Cached:naGB

0,12	Loss:0.05506	Allocated:naGB	Cached:naGB

0,13	Loss:0.04393	Allocated:naGB	Cached:naGB

0,14	Loss:0.05144	Allocated:naGB	Cached:naGB

0,15	Loss:0.05188	Allocated:naGB	Cached:naGB

0,16	Loss:0.05526	Allocated:naGB	Cached:naGB

0,17	Loss:0.03742	Allocated:naGB	Cached:naGB

0,18	Loss:0.04996	Allocated:naGB	Cached:naGB

0,19	Loss:0.04647	Allocated:naGB	Cached:naGB

0,20	Loss:0.04963	Allocated:naGB	Cached:naGB

0,21	Loss:0.04038	Allocated:naGB	Cached:naGB

0,22	Loss:0.04393	Allocated:naGB	Cached:naGB


In [ ]:
torch.cuda.empty_cache()

In [4]:
#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024))

GeForce GTX 1050
Memory Usage:
Allocated: 0.0 GB
Cached:    2048
